# Titanic w/ Pipelines
## Goal
Take what we did in V1 and expand upon it with more info/tutorials on pipelining.

## Relevant Material
[Deep dive in sklearn pipelines](https://www.kaggle.com/baghern/a-deep-dive-into-sklearn-pipelines)  
[Simple pipeline example with scikit learn](https://towardsdatascience.com/a-simple-example-of-pipeline-in-machine-learning-with-scikit-learn-e726ffbb6976)

## Titanic Data
| Variable | Definition | Key |
| ----- | --- | --- |
| survival | Survived or not | 0 = No, 1 = Yes |
| pclass | Ticket class | 1 = 1st, 2 = 2nd, 3 = 3rd |
| sex | Sex | |
| age | Age in years | |
| sibsp | Num of siblings / spouses aboard | |
| parch | Num of parents / children aboard | |
| ticket | Ticket number | |
| fare | Passengar fare | |
| cabin | Cabin number | |
| embarked | Port of embarkation |   C = Cherbourg, Q = Queenstown, S = Southampton |

## Base setup

In [3]:
%reload_ext autoreload
%autoreload 2

# custom helpers
from helpers.helper import get_splits, process_titanic
# data handling
import numpy as np
import pandas as pd
# output
from termcolor import cprint
import matplotlib.pyplot as plt
import seaborn as sns

cprint('All Modules Imported!', 'green')

All Modules Imported!


## Data Import

In [7]:
train_data = pd.read_csv('./data/train.csv', index_col='PassengerId')
test_data = pd.read_csv('./data/test.csv', index_col='PassengerId')

cprint('Data Imported!', 'green')
cprint('Training Data Example:', 'cyan')
display(train_data)

Data Imported!
Training Data Example:


,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S


## Pipelining
### Reading Notes
- For very basic feature engineering, it's good to encapsulate processing logic into functions so it can be reproduced easily.
- For purposes of optimizing and testing models, it's a good idea to create feature selectors that can be used inside a pipeline to apply transformations on single columns. Check out `TextSelector` and `NumericSelector` in helper.py
  - the selectors can used as follows `('selector', TextSelector(key='processed'))`
- Pipelines are built from pipelines. For each processing step, we can create a mini-pipeline that carries out the task/engineering we need.
  - If doing engineering on individual columns, it's important to join the engineered columns back into the dataset. `sklearn.pipeline`'s `FeatureUnion` method can be great for this.

### Basic Engineering and 

In [15]:
numerical_cols = ['Age', 'SibSp', 'Parch', 'Fare']
categorical_cols = ['Pclass', 'Sex', 'Cabin', 'Embarked']
target = 'Survived'

from sklearn.model_selection import train_test_split

processed_data = process_titanic(train_data, categorical_cols, numerical_cols)
# displayprocessed_data)

features = [c for c in train_data.columns.values if c not in ['PassengerId','Survived','Name','Ticket']]

X_train, X_test, y_train, y_test = train_test_split(train_data[features], train_data[target], test_size=0.33, random_state=42)
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Pclass_Sex,Pclass_Cabin,Pclass_Embarked,Sex_Cabin,Sex_Embarked,Cabin_Embarked
PassengerId,,,,,,,,,,,,,,
7,1,male,54.0,0,0,51.8625,E46,S,1_male,1_E46,1_S,male_E46,male_S,E46_S
719,3,male,NaN,0,0,15.5000,NaN,Q,3_male,3_nan,3_Q,male_nan,male_Q,nan_Q
686,2,male,25.0,1,2,41.5792,NaN,C,2_male,2_nan,2_C,male_nan,male_C,nan_C
74,3,male,26.0,1,0,14.4542,NaN,C,3_male,3_nan,3_C,male_nan,male_C,nan_C
883,3,female,22.0,0,0,10.5167,NaN,S,3_female,3_nan,3_S,female_nan,female_S,nan_S
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,3,female,21.0,0,0,7.6500,NaN,S,3_female,3_nan,3_S,female_nan,female_S,nan_S
271,1,male,NaN,0,0,31.0000,NaN,S,1_male,1_nan,1_S,male_nan,male_S,nan_S
861,3,male,41.0,2,0,14.1083,NaN,S,3_male,3_nan,3_S,male_nan,male_S,nan_S


## Memo:
We got some `nan` values above. In the future, we need to/should handle these nan values in processing.